# Predicción de los costos de Casas
## Joaquín Antonio Ramírez Hernández

In [146]:
from scipy import stats
import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

## 1.-Lectura y traducción de datos

### Datos de Training

In [371]:
train1 = pd.read_csv("./house-prices-advanced-regression-techniques/train.csv")
train1.fillna(0,inplace=True)
train1 = train1.replace({'A':float(0),'C (all)':float(1), 'FV':float(2), 'I':float(3),'RH':float(4),'RL':float(5),'RP':float(6),'RM':float(7), 'Grvl':1,'Pave':2})
train1 = train1.replace({'Reg':1, 'IR1':2, 'IR2':3, 'IR3':4,'Lvl':1, 'Bnk':2, 'HLS':3, 'Low':4})
train1 = train1.replace({'AllPub':1, 'NoSewr':2, 'NoSeWa':3, 'ELO':4, 'Inside':1 , 'Corner':2, 'CulDSac':3, 'FR2': 4,'FR3':5})
train1 = train1.replace({'Gtl':1, 'Mod':2, 'Sev':3, 'Blmngtn':1, 'Blueste':2, 'BrDale':3, 'BrkSide':4,'ClearCr':5, 'CollgCr':6, 'Crawfor':7,'Edwards':8,'Gilbert':9,'IDOTRR':10, 'MeadowV':11,	'Mitchel':12, 'Names':13, 'NoRidge':14, 'NPkVill':15, 'NridgHt':16, 'NWAmes':17, 'NAmes':17, 'OldTown':18, 'SWISU':19, 'Sawyer':20, 'SawyerW':21, 'Somerst':22,'StoneBr':23, 'Timber':24, 'Veenker':25})
train1 = train1.replace({'Artery':1,'Feedr':2, 'Norm':3, 'RRNn':4, 'RRAn':5, 'PosN':6,'RRNe':7, 'RRAe':8, '1Fam':1,'2FmCon':2, 'Duplx':3, 'TwhsE':4, 'TwnhsI':5})
train1 = train1.replace({'Flat':1,'Gable':2, 'Gambrel':3, 'Hip':4, 'Mansard':5, 'Shed':6})
train1 = train1.replace({'ClyTile':1,'CompShg':2, 'Membran':3, 'Metal':4, 'Roll':5, 'Tar&Grv':6,'WdShake':7,'WdShngl':8})
train1 = train1.replace({'AsbShng':1,'AsphShn':2, 'BrkComm':3, 'BrkFace':4, 'CBlock':5, 'CemntBd':6,'HdBoard':7,'ImStucc':8,'MetalSd':9,'Other':10, 'Plywood':11, 'PreCast':12, 'Stucco':14, 'VinylSd':15, 'Wd Sdng':16, 'WdShing':17})
train1 = train1.replace({'BrkCmn':1,'BrkFace':2, 'None':4})
train1 = train1.replace({'Ex':1,'Gd':2,' TA':3, 'Fa':4, 'Po':5 })
train1 = train1.replace({'BrkTil':1, 'PConc':2, 'Slab':3, 'Wood':4})
train1 = train1.replace({'Av':1, 'Mn':2, 'No':3})
train1 = train1.replace({'GLQ':1,'ALQ':2,'BLQ':3, 'Rec':4,'LwQ':5, 'Unf':6 })
train1 = train1.replace({'Floor':1,'GasA':2, 'GasW':3, 'Grav':4, 'OthW':5, 'Wall':6,'N':1, 'Y':2, 'P':3, 'SBrkr':1,'FuseA':2,'FuseF':3, 'FuseP':4,'Mix':5})
train1 = train1.replace({'Typ':1,'Min1':3, 'Min2': 4 , 'Maj1': 5, 'Maj2': 6, 'Sev': 7, 'Sal':8})
train1 = train1.replace({'2Types':1, 'Attchd':2, 'Basment':3,'CarPort': 4, 'Detchd':5})
train1 = train1.replace({'Fin':1, 'RFn':2, 'Unf':3})
train1 = train1.replace({'GdPrv':1,'MnPrv':2,'GdWo':3, 'MnWw':4})
train1 = train1.replace({'Elev':1, 'Gar2':2, 'Othr':3, 'Shed':4,'TenC':5})
train1 = train1.replace({'WD':1, 'CWD':2, 'VWD':3, 'New':4, 'COD':5, 'Con':5,'ConLw':6, 'ConLI':7, 'ConLD':8,'Oth':9})
train1 = train1.replace({'Normal':1, 'Abnorml':2, 'AdjLand':3, 'Alloca':4, 'Family':5, 'Partial':6})

### Datos de Testing

In [372]:
test1 = pd.read_csv("./house-prices-advanced-regression-techniques/test.csv")
test1.fillna(0,inplace=True)
test1 = test1.replace({'A':float(0),'C (all)':float(1), 'FV':float(2), 'I':float(3),'RH':float(4),'RL':float(5),'RP':float(6),'RM':float(7), 'Grvl':1,'Pave':2})
test1 = test1.replace({'Reg':1, 'IR1':2, 'IR2':3, 'IR3':4,'Lvl':1, 'Bnk':2, 'HLS':3, 'Low':4})
test1 = test1.replace({'AllPub':1, 'NoSewr':2, 'NoSeWa':3, 'ELO':4, 'Inside':1 , 'Corner':2, 'CulDSac':3, 'FR2': 4,'FR3':5})
test1 = test1.replace({'Gtl':1, 'Mod':2, 'Sev':3, 'Blmngtn':1, 'Blueste':2, 'BrDale':3, 'BrkSide':4,'ClearCr':5, 'CollgCr':6, 'Crawfor':7,'Edwards':8,'Gilbert':9,'IDOTRR':10, 'MeadowV':11,	'Mitchel':12, 'Names':13, 'NoRidge':14, 'NPkVill':15, 'NridgHt':16, 'NWAmes':17, 'NAmes':17, 'OldTown':18, 'SWISU':19, 'Sawyer':20, 'SawyerW':21, 'Somerst':22,'StoneBr':23, 'Timber':24, 'Veenker':25})
test1 = test1.replace({'Artery':1,'Feedr':2, 'Norm':3, 'RRNn':4, 'RRAn':5, 'PosN':6,'RRNe':7, 'RRAe':8, '1Fam':1,'2FmCon':2, 'Duplx':3, 'TwhsE':4, 'TwnhsI':5})
test1 = test1.replace({'Flat':1,'Gable':2, 'Gambrel':3, 'Hip':4, 'Mansard':5, 'Shed':6})
test1 = test1.replace({'ClyTile':1,'CompShg':2, 'Membran':3, 'Metal':4, 'Roll':5, 'Tar&Grv':6,'WdShake':7,'WdShngl':8})
test1 = test1.replace({'AsbShng':1,'AsphShn':2, 'BrkComm':3, 'BrkFace':4, 'CBlock':5, 'CemntBd':6,'HdBoard':7,'ImStucc':8,'MetalSd':9,'Other':10, 'Plywood':11, 'PreCast':12, 'Stucco':14, 'VinylSd':15, 'Wd Sdng':16, 'WdShing':17})
test1 = test1.replace({'BrkCmn':1,'BrkFace':2, 'None':4})
test1 = test1.replace({'Ex':1,'Gd':2,' TA':3, 'Fa':4, 'Po':5 })
test1 = test1.replace({'BrkTil':1, 'PConc':2, 'Slab':3, 'Wood':4})
test1 = test1.replace({'Av':1, 'Mn':2, 'No':3})
test1 = test1.replace({'GLQ':1,'ALQ':2,'BLQ':3, 'Rec':4,'LwQ':5, 'Unf':6 })
test1 = test1.replace({'Floor':1,'GasA':2, 'GasW':3, 'Grav':4, 'OthW':5, 'Wall':6,'N':1, 'Y':2, 'P':3, 'SBrkr':1,'FuseA':2,'FuseF':3, 'FuseP':4,'Mix':5})
test1 = test1.replace({'Typ':1,'Min1':3, 'Min2': 4 , 'Maj1': 5, 'Maj2': 6, 'Sev': 7, 'Sal':8})
test1 = test1.replace({'2Types':1, 'Attchd':2, 'Basment':3,'CarPort': 4, 'Detchd':5})
test1 = test1.replace({'Fin':1, 'RFn':2, 'Unf':3})
test1 = test1.replace({'GdPrv':1,'MnPrv':2,'GdWo':3, 'MnWw':4})
test1 = test1.replace({'Elev':1, 'Gar2':2, 'Othr':3, 'Shed':4,'TenC':5})
test1 = test1.replace({'WD':1, 'CWD':2, 'VWD':3, 'New':4, 'COD':5, 'Con':5,'ConLw':6, 'ConLI':7, 'ConLD':8,'Oth':9})
test1 = test1.replace({'Normal':1, 'Abnorml':2, 'AdjLand':3, 'Alloca':4, 'Family':5, 'Partial':6})


In [748]:
train = pd.DataFrame(data={'x1':train1.SalePrice.to_numpy(),'x2':train1.OverallQual.to_numpy(),'x3':train1.YearBuilt.to_numpy(), 'x4':train1.YearRemodAdd.to_numpy(),'x5':train1.TotalBsmtSF.to_numpy(),'x6':train1.GrLivArea.to_numpy(), 'x7':train1.FullBath.to_numpy(),'x8':train1.TotRmsAbvGrd.to_numpy(),'x9':train1.GarageCars.to_numpy(),'x10': train1.GarageArea.to_numpy()})

In [749]:
corr = train.corr()
corr

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
x1,1.000000,0.790982,0.522897,0.507101,0.613581,0.708624,0.560664,0.533723,0.640409,0.623431
x2,0.790982,1.000000,0.572323,0.550684,0.537808,0.593007,0.550600,0.427452,0.600671,0.562022
x3,0.522897,0.572323,1.000000,0.592855,0.391452,0.199010,0.468271,0.095589,0.537850,0.478954
x4,0.507101,0.550684,0.592855,1.000000,0.291066,0.287389,0.439046,0.191740,0.420622,0.371600
x5,0.613581,0.537808,0.391452,0.291066,1.000000,0.454868,0.323722,0.285573,0.434585,0.486665
x6,0.708624,0.593007,0.199010,0.287389,0.454868,1.000000,0.630012,0.825489,0.467247,0.468997
x7,0.560664,0.550600,0.468271,0.439046,0.323722,0.630012,1.000000,0.554784,0.469672,0.405656
x8,0.533723,0.427452,0.095589,0.191740,0.285573,0.825489,0.554784,1.000000,0.362289,0.337822
x9,0.640409,0.600671,0.537850,0.420622,0.434585,0.467247,0.469672,0.362289,1.000000,0.882475
x10,0.623431,0.562022,0.478954,0.371600,0.486665,0.468997,0.405656,0.337822,0.882475,1.000000


In [750]:
train2 = pd.DataFrame(data={'x2':train1.OverallQual.to_numpy(),'x3':train1.YearBuilt.to_numpy(),'x4':train1.TotalBsmtSF.to_numpy(),'x5':train1.GrLivArea.to_numpy(), 'x6':train1.FullBath.to_numpy(),'x7':train1.TotRmsAbvGrd.to_numpy(),'x8':train1.GarageCars.to_numpy(),'x9': train1.GarageArea.to_numpy()})#
train2 = train2.drop(1459, axis=0)
train2
train = train.drop(1459, axis=0)

In [751]:
test = pd.DataFrame(data={'x2':test1.OverallQual.to_numpy(),'x3':test1.YearBuilt.to_numpy(),'x4':test1.TotalBsmtSF.to_numpy(),'x5':test1.GrLivArea.to_numpy(), 'x6':test1.FullBath.to_numpy(),'x7':test1.TotRmsAbvGrd.to_numpy(),'x8':test1.GarageCars.to_numpy(),'x9': test1.GarageArea.to_numpy()})#

In [752]:
test

,x2,x3,x4,x5,x6,x7,x8,x9
0,5,1961,882.0,896,1,5,1.0,730.0
1,6,1958,1329.0,1329,1,6,1.0,312.0
2,5,1997,928.0,1629,2,6,2.0,482.0
3,6,1998,926.0,1604,2,7,2.0,470.0
4,8,1992,1280.0,1280,2,5,2.0,506.0
...,...,...,...,...,...,...,...,...
1454,4,1970,546.0,1092,1,5,0.0,0.0
1455,4,1970,546.0,1092,1,6,1.0,286.0
1456,5,1960,1224.0,1224,1,7,2.0,576.0
1457,5,1992,912.0,970,1,6,0.0,0.0


### Definición de mi función f

In [1069]:
def f(x):
    return pd.DataFrame(.79*x.x2+.70*x.x5+.62*x.x9**2)

### Regresión lineal

In [1070]:
linear_model = LinearRegression(normalize=False)
linear_model.fit(train2,f(train2))
linear_model.coef_

array([[ 1.73635538e+03, -9.46749074e+01,  1.69681765e+01,
         3.82198481e+00, -2.25390463e+03,  3.36035984e+03,
        -5.74133197e+04,  7.60649156e+02]])

In [1071]:
linear_model.predict(test)

### Regresión Lasso

In [1074]:
lasso_model = Lasso(alpha=1.5,normalize=True, max_iter=1e6)
lasso_model.fit(train2,f(train2))

Lasso(alpha=1.5, max_iter=1000000.0, normalize=True)

In [1075]:
lasso_model.predict(test)

array([396516.89211508,  93917.04002045, 153207.12853951, ...,
       236741.469281  , -99246.24718308, 239991.65854116])